# Purpose

### 2022-02-14
In this notebook I'll select the clusters for the new FPR experiments for Canada, UK, Australia, & India.

Note that this is supposed to be an SFW experiment, so we'll need to filter out subreddits that are `over_18` or rated as `X`.

In one sheet include BOTH subreddit names & subreddit IDs.

---

TODO: Haven't included place logic (e.g., add direction to: city, state, country subreddits.)


### Updates
2022-02-16: 
I [David] will update the QA sheet so that we have standardized columns/format. Otherwise it'll be more work for us to wait for country managers to format things and then standardize them after the fact



# Imports & notebook setup

In [ ]:
%load_ext autoreload
%autoreload 2

# Register bigquery magic (only needed for laptop/local, not colab)
# %load_ext google.cloud.bigquery

In [ ]:
# colab auth for BigQuery
from google.colab import auth, files, drive
auth.authenticate_user()
print('Authenticated')

Authenticated


## Install custom library

### Append google drive path so we can install library from there

In [ ]:
# Attach google drive & import my python utility functions
# if drive.mount() fails, you can also:
#   MANUALLY CLICK ON "Mount Drive"
import sys


g_drive_root = '/content/drive'

try:
    drive.mount(g_drive_root, force_remount=True)
    print('   Authenticated & mounted Google Drive')
    
except Exception as e:
    try:
        drive._mount(g_drive_root, force_remount=True)
        print('   Authenticated & mounted Google Drive')
    except Exception as e:
        print(e)
        raise Exception('You might need to manually mount google drive to colab')

l_paths_to_append = [
    f'{g_drive_root}/MyDrive/Colab Notebooks',

    # need to append the path to subclu so that colab can import things properly
    f'{g_drive_root}/MyDrive/Colab Notebooks/subreddit_clustering_i18n'
]
for path_ in l_paths_to_append:
    if path_ in sys.path:
        sys.path.remove(path_)
    print(f" Appending path: {path_}")
    sys.path.append(path_)

Mounted at /content/drive
   Authenticated & mounted Google Drive
 Appending path: /content/drive/MyDrive/Colab Notebooks
 Appending path: /content/drive/MyDrive/Colab Notebooks/subreddit_clustering_i18n


### Install library

In [ ]:
# install subclu & libraries needed to read parquet files from GCS & spreadsheets
#  make sure to use the [colab] `extra` because it includes colab-specific libraries
module_path = f"{g_drive_root}/MyDrive/Colab Notebooks/subreddit_clustering_i18n/[colab]"

!pip install -e $"$module_path" --quiet

     |████████████████████████████████| 10.1 MB 23.5 MB/s 
     |████████████████████████████████| 14.2 MB 39.2 MB/s 
     |████████████████████████████████| 965 kB 40.7 MB/s 
     |████████████████████████████████| 144 kB 59.2 MB/s 
     |████████████████████████████████| 285 kB 54.3 MB/s 
     |████████████████████████████████| 13.2 MB 39.2 MB/s 
     |████████████████████████████████| 79.9 MB 97 kB/s 
     |████████████████████████████████| 133 kB 57.5 MB/s 
     |████████████████████████████████| 715 kB 42.9 MB/s 
     |████████████████████████████████| 112 kB 48.7 MB/s 
     |████████████████████████████████| 74 kB 3.1 MB/s 
     |████████████████████████████████| 58 kB 4.8 MB/s 
     |████████████████████████████████| 1.1 MB 39.8 MB/s 
     |████████████████████████████████| 79 kB 7.3 MB/s 
     |████████████████████████████████| 180 kB 60.0 MB/s 
     |████████████████████████████████| 146 kB 56.2 MB/s 
     |████████████████████████████████| 596 kB 55.0 MB/s 
     |████████████

## Regular Imports

In [ ]:
import os
from datetime import datetime

from google.cloud import bigquery

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib_venn import venn2_unweighted, venn3_unweighted
from tqdm import tqdm


# auth for google sheets
import gspread
from oauth2client.client import GoogleCredentials


gc = gspread.authorize(GoogleCredentials.get_application_default())

# os.environ['GOOGLE_CLOUD_PROJECT'] = 'data-science-prod-218515'
os.environ['GOOGLE_CLOUD_PROJECT'] = 'data-prod-165221'

[autoreload of cloudpickle failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
ImportError: cannot import name '_should_pickle_by_reference' from 'cloudpickle.cloudpickle' (/usr/local/lib/python3.7/dist-packages/cloudpickle/cloudpickle.py)
]


## Custom imports

In [ ]:
# subclu imports
import subclu
from subclu.utils.eda import (
    setup_logging, counts_describe, value_counts_and_pcts,
    notebook_display_config, print_lib_versions,
    style_df_numeric, reorder_array,
)
from subclu.models.clustering_utils import (
    create_dynamic_clusters,
    convert_distance_or_ab_to_list_for_fpr,
    reshape_df_to_get_1_cluster_per_row,
    get_primary_topic_mix_cols,
)

from subclu.models.reshape_clusters_v041 import (
    keep_only_target_labels,
    get_table_for_optimal_dynamic_cluster_params,
    get_dynamic_cluster_summary,
    _L_MATURE_CLUSTERS_TO_EXCLUDE_FROM_QA_,
)


setup_logging()
notebook_display_config()
print_lib_versions([gspread, pd, np])

python		v 3.7.12
===
gspread		v: 4.0.1
pandas		v: 1.3.5
numpy		v: 1.21.5


# Checklist to re-run for a country:

- change country name in geo-relevance query
- change country name for google sheet name
    - add google sheet KEY (after created)

# Load data from BigQuery

## Load subreddit geo-relevance & cultural relevance metadata

This data is already in bigQuery so read it straight from there. We'll use it to filter out geo-relevant (German) subs.

Also add the latest ratings so that we can filter based on those.

English-speaking countries don't have ambassador subs right now, so we should be able to create a standard template and replace the country name for these queries.

### SQL geo & cultural

In [ ]:
%%time
%%bigquery df_geo_and_lang --project data-science-prod-218515 

-- Select geo+cultural subreddits for a target country
--  And add latest rating & over_18 flags to exclude X-rated & over_18
DECLARE TARGET_COUNTRY STRING DEFAULT 'France';


SELECT
    s.* EXCEPT(over_18, pt, verdict) 
    , nt.rating_name
    , nt.primary_topic
    , nt.rating_short
    , slo.over_18
    , slo.allow_discovery
    , CASE 
        WHEN(COALESCE(slo.over_18, 'f') = 't') THEN 'over_18_or_X_M_D_V'
        WHEN(COALESCE(nt.rating_short, '') IN ('X', 'M', 'D', 'V')) THEN 'over_18_or_X_M_D_V'
        ELSE 'unrated_or_E'
    END AS grouped_rating

FROM `reddit-employee-datasets.david_bermejo.subclu_v0041_subreddit_clusters_c_a` AS t
    -- Inner join b/c we only want to keep subs that are geo-relevant AND in topic model
    INNER JOIN (
        SELECT *
        FROM `reddit-employee-datasets.david_bermejo.subclu_subreddit_geo_score_standardized_20220212`
        WHERE country_name = TARGET_COUNTRY
    ) AS s
        ON t.subreddit_id = s.subreddit_id

    -- Add rating so we can get an estimate for how many we can actually use for recommendation
    LEFT JOIN (
        SELECT *
        FROM `data-prod-165221.ds_v2_postgres_tables.subreddit_lookup`
        -- Get latest partition
        WHERE dt = DATE(CURRENT_DATE() - 2)
    ) AS slo
    ON s.subreddit_id = slo.subreddit_id
    LEFT JOIN (
        SELECT * FROM `data-prod-165221.cnc.shredded_crowdsource_topic_and_rating`
        WHERE pt = DATE(CURRENT_DATE() - 2)
    ) AS nt
        ON s.subreddit_id = nt.subreddit_id

    -- Exclude popular US subreddits
    -- Can't query this table from local notebook because of errors getting google drive permissions. smh, excludefor now
    LEFT JOIN `reddit-employee-datasets.david_bermejo.subclu_subreddits_top_us_to_exclude_from_relevance` tus
        ON s.subreddit_name = LOWER(tus.subreddit_name)

WHERE 1=1
    AND s.subreddit_name != 'profile'
    AND COALESCE(s.type, '') = 'public'
    AND COALESCE(s.verdict, 'f') <> 'admin_removed'
    AND COALESCE(slo.over_18, 'f') = 'f'
    AND COALESCE(nt.rating_short, '') NOT IN ('X', 'D')

    AND(
        s.geo_relevance_default = TRUE
        OR s.relevance_percent_by_subreddit = TRUE
        OR s.relevance_percent_by_country_standardized = TRUE
    )
    AND country_name IN (
            TARGET_COUNTRY
        )

    AND (
         -- Exclude subs that are top in US but we want to exclude as culturally relevant
         --  For simplicity, let's go with the English exclusion (more relaxed) than the non-English one
         COALESCE(tus.english_exclude_from_relevance, '') <> 'exclude'
    )

ORDER BY e_users_percent_by_country_standardized DESC, users_l7 DESC, subreddit_name
;

CPU times: user 409 ms, sys: 35.5 ms, total: 444 ms
Wall time: 10.8 s


### Check df with geo + language information

In [ ]:
print(df_geo_and_lang.shape)

(378, 26)


In [ ]:
df_geo_and_lang.iloc[:4, :9]

,subreddit_id,subreddit_name,country_name,geo_relevance_default,b_users_percent_by_subreddit,e_users_percent_by_country_standardized,c_users_percent_by_country,d_users_percent_by_country_rank,relevance_percent_by_subreddit
0,t5_2um6i,rance,France,True,0.720740,10.472597,0.021637,3,True
1,t5_2qkoi,paris,France,True,0.684907,10.416217,0.008868,25,True
2,t5_2qhjz,france,France,True,0.683145,10.029642,0.052882,1,True
3,t5_2zkfk,askfrance,France,True,0.828429,9.922256,0.006089,51,True


## Load model labels (clusters)

The clusters now live in a Big Query table and have standardized names, so pull the data from there.

### SQL labels


In [ ]:
%%time
%%bigquery df_labels --project data-science-prod-218515 

-- select subreddit clusters from bigQuery

SELECT
    sc.subreddit_id
    , sc.subreddit_name
    , nt.primary_topic

    , sc.* EXCEPT(subreddit_id, subreddit_name, primary_topic_1214)
FROM `reddit-employee-datasets.david_bermejo.subclu_v0041_subreddit_clusters_c_a` sc
    LEFT JOIN (
        -- New view should be visible to all, but still comes from cnc_taxonomy_cassandra_sync
        SELECT * FROM `data-prod-165221.cnc.shredded_crowdsource_topic_and_rating`
        WHERE DATE(pt) = (CURRENT_DATE() - 2)
    ) AS nt
        ON sc.subreddit_id = nt.subreddit_id
;

CPU times: user 9.82 s, sys: 543 ms, total: 10.4 s
Wall time: 24.8 s


### Check label outputs

In [ ]:
print(df_labels.shape)
df_labels.iloc[:4, :9]

(49558, 51)


,subreddit_id,subreddit_name,primary_topic,model_sort_order,posts_for_modeling_count,k_0013_label,k_0023_label,k_0041_label,k_0059_label
0,t5_5a9iie,progonlydj,None,40079,1000,12,19,34,49
1,t5_2x9c7,googleplaymusic,Music,40080,31,12,19,34,49
2,t5_3jzsk,ravedj,Music,40081,1000,12,19,34,49
3,t5_2rgie,happyhardcore,Music,40082,152,12,19,34,49


In [ ]:
counts_describe(df_labels.iloc[:, :9])

,dtype,count,unique,unique-percent,null-count,null-percent
subreddit_id,object,"49,558","49,558",100.00%,0,0.00%
subreddit_name,object,"49,558","49,558",100.00%,0,0.00%
primary_topic,object,"40,714",52,0.13%,"8,844",17.85%
model_sort_order,int64,"49,558","49,558",100.00%,0,0.00%
posts_for_modeling_count,int64,"49,558",999,2.02%,0,0.00%
k_0013_label,int64,"49,558",13,0.03%,0,0.00%
k_0023_label,int64,"49,558",23,0.05%,0,0.00%
k_0041_label,int64,"49,558",41,0.08%,0,0.00%
k_0059_label,int64,"49,558",59,0.12%,0,0.00%


# Reshape data
Apply reshaping fxns so that we can export the data in a format that's good for QA.

## Keep only labels for Target subreddits


In [ ]:
%%time
df_labels_target = keep_only_target_labels(
    df_labels=df_labels,
    df_geo=df_geo_and_lang,
    col_sort_order='model_sort_order',
    l_ix_subs=['subreddit_id', 'subreddit_name'],
    l_cols_to_front=None,
    geo_cols_to_drop=None,
)

0 <- subs to drop b/c they're not in model
(378, 74) <- df_labels_target.shape
CPU times: user 184 ms, sys: 6.74 ms, total: 191 ms
Wall time: 196 ms


In [ ]:
counts_describe(df_labels_target.iloc[:, :15])

,dtype,count,unique,unique-percent,null-count,null-percent
model_sort_order,int64,378,378,100.00%,0,0.00%
subreddit_id,object,378,378,100.00%,0,0.00%
subreddit_name,object,378,378,100.00%,0,0.00%
primary_topic,object,274,41,14.96%,104,27.51%
rating_short,object,327,2,0.61%,51,13.49%
over_18,object,86,1,1.16%,292,77.25%
rating_name,object,327,2,0.61%,51,13.49%
posts_for_modeling_count,int64,378,196,51.85%,0,0.00%
k_0013_label,int64,378,12,3.17%,0,0.00%
k_0023_label,int64,378,22,5.82%,0,0.00%


## Run loop to find "optimal" min_num of subreddits for dynamic clusters


We want to balance two things:
- prevent orphan subreddits
- prevent clusters that are too large to be meaningful

In order to do this at a country level, we'll be better off starting with smallest cluster size and roll up until we have at least N subreddits in one cluster.

Find optimal `min_subreddits_in_cluster` based on:
- `orphan count`, 
- `number of clusters`,
- & other info

number might be different for each country and even within a country it might differ by when we filter NSFW subs.

### Loop

In [ ]:
%%time

col_new_cluster_val = 'cluster_label'
col_new_cluster_name = 'cluster_label_k'
col_new_cluster_prim_topic = 'cluster_majority_primary_topic'
col_new_cluster_topic_mix = 'cluster_topic_mix'

df_optimal_min_check = get_table_for_optimal_dynamic_cluster_params(
        df_labels_target=df_labels_target,
        col_new_cluster_val=col_new_cluster_val,
        col_new_cluster_name=col_new_cluster_name,
        col_new_cluster_prim_topic=col_new_cluster_prim_topic,
        col_new_cluster_topic_mix=col_new_cluster_topic_mix,
        min_subs_in_cluster_list=np.arange(3, 11),
        verbose=False,
)

100%|██████████| 16/16 [00:00<00:00, 38.32it/s]

100%|██████████| 16/16 [00:00<00:00, 35.33it/s]

100%|██████████| 16/16 [00:00<00:00, 39.25it/s]

100%|██████████| 16/16 [00:00<00:00, 38.03it/s]

100%|██████████| 16/16 [00:00<00:00, 38.92it/s]

100%|██████████| 16/16 [00:00<00:00, 34.44it/s]

100%|██████████| 16/16 [00:00<00:00, 33.37it/s]

100%|██████████| 16/16 [00:00<00:00, 33.97it/s]

100%|██████████| 8/8 [00:07<00:00,  1.11it/s]


CPU times: user 7.03 s, sys: 222 ms, total: 7.25 s
Wall time: 7.37 s


### Display loop results

In [ ]:
def highlight_below_threshold(val, threshold=1):
    if val <= threshold:
        return "color:purple; font-weight: bold; background-color:yellow;"
    else:
        return ''

col_num_orph_subs = 'num_orphan_subreddits'
# col_num_subs_mean = 'num_subreddits_per_cluster_mean'
col_num_subs_median = 'num_subreddits_per_cluster_median'

style_df_numeric(
    df_optimal_min_check,
    rename_cols_for_display=True,
    l_bar_simple=[col_num_orph_subs,
                  col_num_subs_median,]
).applymap(highlight_below_threshold, subset=[col_num_orph_subs.replace('_', ' ')])


,subs to cluster count,min subreddits in cluster,cluster count,num orphan subreddits,num subreddits per cluster mean,num subreddits per cluster median,num clusters with mature primary topic,cluster ids with orphans
0,378,3,78,1,4.85,4.00,13,0005
1,378,4,64,3,5.91,6.00,10,"0001, 0005, 0012"
2,378,5,53,2,7.13,7.00,9,"0005, 0011"
3,378,6,48,1,7.88,8.00,8,0005
4,378,7,45,1,8.40,8.00,9,0005
5,378,8,41,1,9.22,9.00,8,0005
6,378,9,36,2,10.50,11.00,8,"0001, 0005"
7,378,10,34,1,11.12,12.00,7,0005


In [ ]:
del df_optimal_min_check

## Get dyanimc clusters (apply optimal num from above)

side bar: about 57% of subreddits in Australia only had a single primary topic as their `topic_mix`, so combining `primary topic` might not give us as much info as we hoped.

At the same time, for 43% of subs we might get additional detail by combining the primary topics.


In [ ]:
n_min_subs_in_cluster_optimal = 6

n_mix_start = 4
l_ix = ['subreddit_id', 'subreddit_name']
col_new_cluster_topic_mix = 'cluster_topic_mix'
col_subreddit_topic_mix = 'subreddit_full_topic_mix'
col_full_depth_mix_count = 'subreddit_full_topic_mix_count'
suffix_new_topic_mix = '_topic_mix_nested'

# even if cluster at k < 20 is generic, keep it to avoid orphan subs
l_cols_labels = (
    [c for c in df_labels_target.columns
        if all([c != col_new_cluster_val, c.endswith('_label')])
        ]
)

df_labels_target_dynamic_raw = create_dynamic_clusters(
    df_labels_target,
    agg_strategy='aggregate_small_clusters',
    min_subreddits_in_cluster=n_min_subs_in_cluster_optimal,
    l_cols_labels_input=l_cols_labels,
    col_new_cluster_val=col_new_cluster_val,
    col_new_cluster_name=col_new_cluster_name,
    col_new_cluster_prim_topic=col_new_cluster_prim_topic,
    n_mix_start=n_mix_start,
    col_new_cluster_topic_mix=col_new_cluster_topic_mix,
    col_subreddit_topic_mix=col_subreddit_topic_mix,
    col_full_depth_mix_count=col_full_depth_mix_count,
    suffix_new_topic_mix=suffix_new_topic_mix,
    l_ix=l_ix,
    verbose=True,
)

01:01:37 | INFO | "Concat'ing nested cluster labels..."
01:01:38 | INFO | "Getting topic mix at different depths..."
01:01:38 | INFO | "  Assigning base topic mix cols"
01:01:38 | INFO | "  Creating deepest base topic mix col..."
01:01:38 | INFO | "  Iterating through additional subs with multiple topics..."
100%|██████████| 16/16 [00:00<00:00, 18.05it/s]
01:01:39 | INFO | "Initializing values for strategy: aggregate_small_clusters"
01:01:39 | INFO | "  Looping to roll-up clusters from smallest to largest..."
100%|██████████| 21/21 [00:00<00:00, 77.18it/s]
01:01:39 | INFO | "(378, 124) <- output shape"


In [ ]:
# [c for c in df_labels_target_dynamic_raw.columns if 'nested' in c]

In [ ]:
style_df_numeric(
    get_dynamic_cluster_summary(
        df_labels_target_dynamic_raw,
        return_dict=False,
    ),
    rename_cols_for_display=True,
)

,cluster count,num orphan subreddits,num subreddits per cluster mean,num subreddits per cluster median,num clusters with mature primary topic,cluster ids with orphans
0,48,1,7.88,8.00,8,0005


In [ ]:
# # check column order
style_df_numeric(
    df_labels_target_dynamic_raw.iloc[104:109, :15],
    rename_cols_for_display=True,
    int_labels=['total_users_in', 'num_of_countries_', 'users_in_subreddit_from_country_l28',
                    'by_country_rank',
                    ],
    pct_cols=['b_users_percent_by_subreddit',
                  'c_users_percent_by_country',
                  'users_percent_by_country_avg',
                  ],
    pct_labels='',
)

,subreddit id,subreddit name,cluster topic mix,primary topic,rating short,subreddit full topic mix,rating name,over 18,geo relevance default,relevance percent by subreddit,relevance percent by country standardized,b users percent by subreddit,e users percent by country standardized,d users percent by country rank,model sort order
104,t5_ft3oi,streetviewcarspotting,Gaming,Cars and Motor Vehicles,E,Gaming | Technology,Everyone,f,False,False,True,7.04%,2.47,"26,211","20,777"
105,t5_3jxwi,couturereps,Fashion,Fashion,E,Fashion,Everyone,-,False,False,True,3.97%,2.37,"5,914","21,288"
106,t5_4x1pmi,frenchreps,Fashion,Fashion,E,Fashion,Everyone,-,True,True,True,46.41%,5.18,"2,907","21,292"
107,t5_57d4md,frenchyreps,Fashion,-,E,Fashion,Everyone,-,True,True,True,56.21%,4.48,"2,531","21,293"
108,t5_48doco,fatch,Fashion,Fashion,E,Fashion,Everyone,-,True,True,False,82.66%,1.38,"7,531","21,304"


### Minor QA checks

In [ ]:
# # check column order
# style_df_numeric(
#     df_labels_target_dynamic_raw.iloc[70:74, -22:],
#     rename_cols_for_display=True,
#     int_labels=['total_users_in', 'num_of_countries_', 'users_in_subreddit_from_country_l28',
#                     'by_country_rank',
#                     ],
#     pct_cols=['b_users_percent_by_subreddit',
#                   'c_users_percent_by_country',
#                   'users_percent_by_country_avg',
#                   ],
#     pct_labels='',
# )

In [ ]:
value_counts_and_pcts(
    df_labels_target_dynamic_raw[col_new_cluster_topic_mix],
    top_n=9,
)

,cluster_topic_mix-count,cluster_topic_mix-percent,cluster_topic_mix-pct_cumulative_sum
Gaming,93,24.6%,24.6%
Mature Themes and Adult Content,48,12.7%,37.3%
Music,34,9.0%,46.3%
Technology,27,7.1%,53.4%
Politics,19,5.0%,58.5%
Place,18,4.8%,63.2%
Internet Culture and Memes,15,4.0%,67.2%
Crypto,13,3.4%,70.6%
Animals and Pets,13,3.4%,74.1%


In [ ]:
# how many final clusters have multiple topics?
value_counts_and_pcts(
    df_labels_target_dynamic_raw[col_new_cluster_topic_mix].str.count('\|')
)

,cluster_topic_mix-count,cluster_topic_mix-percent,cluster_topic_mix-pct_cumulative_sum
0,348,92.1%,92.1%
1,22,5.8%,97.9%
3,8,2.1%,100.0%


In [ ]:
# how many SUBREDDITS have multiple topics? (when we check the deepest clusters)
#  these two calls are equivalent

# value_counts_and_pcts(
#     df_labels_target_dynamic_raw[col_subreddit_topic_mix].str.count('\|')
# )

value_counts_and_pcts(
    df_labels_target_dynamic_raw[col_full_depth_mix_count]
)

,subreddit_full_topic_mix_count-count,subreddit_full_topic_mix_count-percent,subreddit_full_topic_mix_count-pct_cumulative_sum
1,247,65.3%,65.3%
2,69,18.3%,83.6%
3,38,10.1%,93.7%
5,18,4.8%,98.4%
4,6,1.6%,100.0%


In [ ]:
style_df_numeric(
    df_labels_target_dynamic_raw
    [df_labels_target_dynamic_raw[col_full_depth_mix_count] >= 5]
    .iloc[-5:, :9]
    ,
    rename_cols_for_display=True,
)

,subreddit id,subreddit name,cluster topic mix,primary topic,rating short,subreddit full topic mix,rating name,over 18,geo relevance default
214,t5_2twr5,corsica,History,-,E,"History | Place | Learning and Education | Culture, Race, and Ethnicity | Hobbies",Everyone,-,False
215,t5_2woqv,occitan,History,-,E,"History | Place | Learning and Education | Culture, Race, and Ethnicity | Hobbies",Everyone,-,False
225,t5_37nlj,geographie,History,Learning and Education,E,History | Place | Funny/Humor | Art | Learning and Education,Everyone,-,True
232,t5_3lw9qz,redpacketsecurity,Science,Technology,E,"Science | Mature Themes and Adult Content | Music | Business, Economics, and Finance | Technology",Everyone,-,False
376,t5_wst9p,annuaire,Music,-,E,Music | Mature Themes and Adult Content | Internet Culture and Memes | Meta/Reddit | Learning and Education,Everyone,f,True


## Re-assign orphan subreddits (optional)

If there are subreddits that are orphan (see summary above), check them out to see if we can re-assign them w/o too much work. if we can't skip and move to the next country.

In [ ]:
# # check subs around orphan sub
# # n_plus_minus_ = 5
# ix_orphan_ = (
#     df_labels_target_dynamic_raw
#     [df_labels_target_dynamic_raw[col_new_cluster_val] == '0011']
#     .index
# )
# df_labels_target_dynamic_raw.iloc[ix_orphan_, :9]

In [ ]:
# # check other subs that are in the same cluster as orphan sub (at broadest level)
# l_cols_orphan_check = (
#     [
#         'subreddit_id',
#         col_new_cluster_topic_mix, 
#         # col_new_cluster_val,  # this can be really long and makes comparing harder
#         # col_subreddit_topic_mix,
#         'subreddit_name', 
#         col_new_cluster_name
#     ] +
#     l_cols_labels[:-5]
# )

# style_df_numeric(
#     df_labels_target_dynamic_raw
#     [df_labels_target_dynamic_raw['k_0013_label'] == 11]
#     [l_cols_orphan_check]
#     .iloc[3:14, :50]
#     ,
#     l_bar_simple=[c for c in l_cols_orphan_check[4:] if c.endswith('_label')],
#     rename_cols_for_display=True,

# )

In [ ]:
# label_k_to_reassign_ = 'k_0320_label'
# label_val_to_reassign_ = '0011-0018-0032-0043-0046-0058-0062-0087-0244'
# subreddit_id_orphan_ = 't5_2tt7r'

# mask_orphan_and_new_group = (
#     (df_labels_target_dynamic_raw['subreddit_id'] == subreddit_id_orphan_) |
#     (
#         (df_labels_target_dynamic_raw[col_new_cluster_name] == label_k_to_reassign_) &
#         (df_labels_target_dynamic_raw[col_new_cluster_val] == label_val_to_reassign_)
#     )
# )

# # assign is similar to what we do in the dynamic function
# label_k_new_ = 'k_0118_label'
# label_val_new_col_ = f"{label_k_new_}_nested"
# new_prim_topic_col_ = label_k_new_.replace('_label', '_majority_primary_topic')
# c_update_topic_mix_ = label_k_new_.replace('_label', suffix_new_topic_mix)

# df_labels_target_dynamic_raw.loc[
#     mask_orphan_and_new_group,
#     col_new_cluster_name
# ] = label_k_new_

# df_labels_target_dynamic_raw.loc[
#     mask_orphan_and_new_group,
#     col_new_cluster_val
# ] = df_labels_target_dynamic_raw[mask_orphan_and_new_group][label_val_new_col_]

# df_labels_target_dynamic_raw.loc[
#     mask_orphan_and_new_group,
#     col_new_cluster_prim_topic
# ] = df_labels_target_dynamic_raw[mask_orphan_and_new_group][new_prim_topic_col_]

# df_labels_target_dynamic_raw.loc[
#     mask_orphan_and_new_group,
#     col_new_cluster_topic_mix
# ] = df_labels_target_dynamic_raw[mask_orphan_and_new_group][c_update_topic_mix_]

# del mask_orphan_and_new_group, label_k_to_reassign_, label_val_to_reassign_
# del label_k_new_, label_val_new_col_, new_prim_topic_col_

In [ ]:
# check again, num of orphans should be lower than before
style_df_numeric(
    get_dynamic_cluster_summary(
        df_labels_target_dynamic_raw,
        return_dict=False,
    ),
    rename_cols_for_display=True,
)

,cluster count,num orphan subreddits,num subreddits per cluster mean,num subreddits per cluster median,num clusters with mature primary topic,cluster ids with orphans
0,48,1,7.88,8.00,8,0005


In [ ]:
value_counts_and_pcts(
    df_labels_target_dynamic_raw,
    ['cluster_label'],
    top_n=None,
    return_df=True
)['count'].describe()

count    48.000000
mean      7.875000
std       2.833125
min       1.000000
25%       7.000000
50%       8.000000
75%       9.000000
max      17.000000
Name: count, dtype: float64

## Get cluster for humans (list of subs in a cluster in a cell)
Here we get 1 cluster per row. 
Use cases:
- It makes it easier to quickly check NSFW clusters that we'll filter out
- we'll append the list of subreddit names from here to the final table for QA (makes it easier to evaluate whether the cluster makes sense).


In [ ]:
col_subs_in_cluster_count = 'subs_in_cluster_count'
col_list_cluster_names = 'list_cluster_subreddit_names'

df_cluster_for_humans = reshape_df_to_get_1_cluster_per_row(
    df_labels_target_dynamic_raw,
    col_counterpart_count=col_subs_in_cluster_count,
    col_list_cluster_names=col_list_cluster_names,
    col_list_cluster_ids='list_cluster_subreddit_ids',
    col_new_cluster_val=col_new_cluster_val,
    col_new_cluster_name=col_new_cluster_name,
    col_new_cluster_topic=col_new_cluster_topic_mix,
    verbose=False,
    get_one_column_per_sub_id=False,
)

(48, 6)


In [ ]:
df_cluster_for_humans.iloc[40:45, 1:]

,cluster_label_k,cluster_topic_mix,subs_in_cluster_count,list_cluster_subreddit_ids,list_cluster_subreddit_names
40,k_0118_label,Gaming,7,"t5_30o52, t5_2u3f5, t5_y2v8o, t5_592zr9, t5_3f1pi, t5_2s9ql, t5_384a7","shareyoursheps, thewitness, solarash, chorusvideogame, zeldabotw, mariokart, dbzdokkanbattle"
41,k_0118_label,Gaming,13,"t5_2qtbr, t5_3gcb7, t5_2xkyv, t5_39cf6, t5_2x9xjy, t5_1mm1pt, t5_56o22l, t5_31su6g, t5_2xuj2, t5_2upxx, t5_39umt, t5_aaoo1, t5_3ejda","streaming, twitchfr, leagueoflegendsfr, rocketleagueesports, ridersrepublic, firstclasstrouble, propnight, dressuptimeprincess, outerwilds, zombicide, swgalaxyofheroes, celestegame, foxholegame"
42,k_0013_label,Music,11,"t5_2qz0k, t5_2qtn5, t5_2uab4, t5_2tnz5, t5_lo39g, t5_3oou3, t5_4rjn2y, t5_35ged9, t5_5ahutt, t5_2t37n, t5_2timi","muse, daftpunk, justicemusic, edbangerrecords, squeezie, orelsan, giscardpunk, sciencefiction_fr, culturepop, niptuck, kaamelott"
43,k_0320_label,Music,8,"t5_3mga1, t5_2rieb, t5_2ytof, t5_2tgto, t5_2sdgb, t5_2sj6d, t5_2wraz, t5_2rmsp","drumandbassproduction, mymusic, 667, frenchrap, ecouteca, jamiroquai, musiquefrancaise, gamemusic"
44,k_0013_label,Animals and Pets,13,"t5_39bx1, t5_2vcw2, t5_4s8ass, t5_s3lnd, t5_x58ta, t5_37dfwj, t5_46ltf9, t5_3d2lu5, t5_ni6v1, t5_ne3w4, t5_31j6z, t5_3k00t, t5_2r8ok","jardin, rubberducks, weirdlikeme, okcopainattard, gendarmerienationale, cyrilmp4, trouduction, drole, animefr, mangafr, gunnm, petscop, photographie"


In [ ]:
# df_cluster_for_humans.tail(9)

### Check subs that have mature topics

Make a list of sub IDs to exclude for clean df

In [ ]:
mask_mature_clusters_ = (
    df_cluster_for_humans[col_new_cluster_topic_mix].str.lower()
    .str.contains('mature')
)
print(mask_mature_clusters_.sum())

9


In [ ]:
(
    df_cluster_for_humans
    [mask_mature_clusters_]
    .iloc[:11, :]
)

,cluster_label,cluster_label_k,cluster_topic_mix,subs_in_cluster_count,list_cluster_subreddit_ids,list_cluster_subreddit_names,exclude_from_qa
0,0001,k_0013_label,Mature Themes and Adult Content,3,"t5_3wcby1, t5_2uhed, t5_53sdsa","influenceusestvr, alyssamilano, edithbhr",exclude from QA
1,0001-0001-0002-0002-0002-0003-0003-0003,k_0118_label,Mature Themes and Adult Content,10,"t5_2oq21o, t5_4148ft, t5_5blr1t, t5_4fknoz, t5_586svs, t5_3zhcr1, t5_304tsk, t5_56g6um, t5_4otz42, t5_4gm7ij","louvecchia, creamyjuicy, ahloubaby, emilycocea, mym_antille_guyane, paolalocatelli, 310lexa, 972_nude, melaniemartialdc, anaisbragadias",exclude from QA
2,0001-0001-0003-0003-0003-0004-0005-0005,k_0118_label,Mature Themes and Adult Content,9,"t5_3kvhsq, t5_454q61, t5_4b02ee, t5_4j49yz, t5_54ur0m, t5_58rhek, t5_4dp3eb, t5_5ft7pk, t5_298xh4","lolaclark, lesmissilles, akirayass, vaninababyy, agatheauprouxfr, sara_irtg, jessica__belkin, jacquieetmichel, irismittenaere",exclude from QA
3,0002,k_0013_label,Mature Themes and Adult Content,5,"t5_5ao59m, t5_4ypstc, t5_54tdnc, t5_4zttbn, t5_2ewz28","pidifiak, pinkgeek, unmissablnsfw, nudesfrance, juliacourtes",exclude from QA
4,0002-0002-0005-0005-0005,k_0063_label,Mature Themes and Adult Content,11,"t5_4uivf6, t5_3qthjb, t5_4zo2r1, t5_550egb, t5_34xilk, t5_4h70rf, t5_5a49a2, t5_22b154, t5_3frwa2, t5_3wmkp1, t5_506nuu","frenchgayvideo, supernovaez, leanedsn, youtubeuse_fr_, evelyneyontchev, nadegelacroixmym, enyaaoff_leak, audreyshanice, mymfansmodels, juliabayoneta, yuboporn",exclude from QA
5,0002-0002-0005-0005-0005-0007-0008-0009,k_0118_label,Mature Themes and Adult Content,7,"t5_5djexe, t5_505os1, t5_4nh28f, t5_2mcpzf, t5_41l9ex, t5_4rgn0o, t5_4hpc1f","rromaaaanetiktok, jaddelf, tiffanyanaa, danaebessin, nathalieandreani, youtubeusessexy, morenaaluna",exclude from QA
6,0004,k_0013_label,Mature Themes and Adult Content,2,"t5_2zzve, t5_32b5w","socialparis, r4rfrance",exclude from QA
7,0005,k_0013_label,Mature Themes and Adult Content,1,t5_3j2k1,candidshiny,exclude from QA
22,0008-0013-0023-0032-0034-0044-0046-0064,k_0118_label,Mature Themes and Adult Content | Gender,7,"t5_2ankda, t5_pmpp9, t5_38h91, t5_2tg5x, t5_2tm6f, t5_3bust, t5_2rvavq","joyeuxfrerots, lgbtqietplus, transgenre, feminisme, askredddit, tinderfrance, advicesnark",exclude from QA


In [ ]:
# (
#     df_cluster_for_humans
#     [mask_mature_clusters_]
#     .iloc[-12:, :]
# )

- model did a good job identifying this drug cluster... but yeah def don't want to increase their reach   
    - `0008-0014-0025-0034-0036-0046-0048-0066`
    - `0008-0014-0025-0034-0036-0046-0048-0066-0190-0378`

- the drinking might be SFW... but prob not the 420
    - `0008-0014-0025-0034-0036-0047-0049-0067`
    - `0008-0014-0025-0034-0036-0047-0049-0067-0193`

In [ ]:
# might need to re-import in order for updates to show up
from subclu.models.reshape_clusters_v041 import _L_MATURE_CLUSTERS_TO_EXCLUDE_FROM_QA_

_L_MATURE_CLUSTERS_TO_EXCLUDE_FROM_QA_

['0001',
 '0001-0001-0001-0001-0001-0002-0002-0002',
 '0001-0001-0002-0002-0002-0003-0003-0003',
 '0001-0001-0003-0003-0003-0004-0005-0005',
 '0002',
 '0002-0002',
 '0002-0002-0005-0005-0005',
 '0002-0002-0005-0005-0005-0007-0008-0009',
 '0004',
 '0005',
 '0005-0007-0012',
 '0008-0013',
 '0008-0014-0025-0034-0036-0046-0048-0066',
 '0008-0014-0025-0034-0036-0046-0048-0066-0190-0378',
 '0008-0014-0025-0034-0036-0047-0049-0067',
 '0008-0014-0025-0034-0036-0047-0049-0067-0193',
 '0010-0017-0030-0040-0042-0053-0056-0078-0219-0437-0630-0705',
 '0010-0017-0030-0040-0042-0053-0056-0078-0219-0437-0630-0705-1017-1195-1439-1532-1828-2032-2196-2380-2482-2516']

### Add the flag to exclude from QA & the list of sub names to df-raw

In [ ]:
val_exclude_from_qa_ = 'exclude from QA'
col_exclude_from_qa = 'exclude_from_qa'

df_cluster_for_humans[col_exclude_from_qa] = np.where(
    df_cluster_for_humans[col_new_cluster_val].isin(_L_MATURE_CLUSTERS_TO_EXCLUDE_FROM_QA_),
    val_exclude_from_qa_,
    'keep',  
)
display(value_counts_and_pcts(df_cluster_for_humans[col_exclude_from_qa]))


df_labels_target_dynamic_raw = (
    df_labels_target_dynamic_raw
    # drop in case we're redoing the values
    .drop(
        [c for c in [col_list_cluster_names, col_subs_in_cluster_count, col_exclude_from_qa] if c in df_labels_target_dynamic_raw.columns],
        axis=1
    )
    .merge(
        df_cluster_for_humans[[col_new_cluster_val, col_list_cluster_names, col_subs_in_cluster_count, col_exclude_from_qa]],
        how='left',
        on=[col_new_cluster_val],
    )
)

df_labels_target_dynamic_raw.shape

,exclude_from_qa-count,exclude_from_qa-percent,exclude_from_qa-pct_cumulative_sum
keep,40,83.3%,83.3%
exclude from QA,8,16.7%,100.0%


(378, 131)

In [ ]:
df_labels_target_dynamic_raw.iloc[:5, -9:]

,k_2830_topic_mix_nested,k_3145_topic_mix_nested,k_3411_topic_mix_nested,k_3706_topic_mix_nested,k_3864_topic_mix_nested,k_3927_topic_mix_nested,list_cluster_subreddit_names,subs_in_cluster_count,exclude_from_qa
0,Mature Themes and Adult Content | Celebrity,Mature Themes and Adult Content | Celebrity,Mature Themes and Adult Content | Celebrity,Mature Themes and Adult Content | Celebrity,Mature Themes and Adult Content | Celebrity,Mature Themes and Adult Content | Celebrity,"influenceusestvr, alyssamilano, edithbhr",3,exclude from QA
1,Mature Themes and Adult Content,Mature Themes and Adult Content,Mature Themes and Adult Content,Mature Themes and Adult Content,Mature Themes and Adult Content,Mature Themes and Adult Content,"louvecchia, creamyjuicy, ahloubaby, emilycocea, mym_antille_guyane, paolalocatelli, 310lexa, 972_nude, melaniemartialdc, anaisbragadias",10,exclude from QA
2,Mature Themes and Adult Content,Mature Themes and Adult Content,Mature Themes and Adult Content,Mature Themes and Adult Content,Mature Themes and Adult Content,Mature Themes and Adult Content,"louvecchia, creamyjuicy, ahloubaby, emilycocea, mym_antille_guyane, paolalocatelli, 310lexa, 972_nude, melaniemartialdc, anaisbragadias",10,exclude from QA
3,Mature Themes and Adult Content,Mature Themes and Adult Content,Mature Themes and Adult Content,Mature Themes and Adult Content,Mature Themes and Adult Content,Mature Themes and Adult Content,"louvecchia, creamyjuicy, ahloubaby, emilycocea, mym_antille_guyane, paolalocatelli, 310lexa, 972_nude, melaniemartialdc, anaisbragadias",10,exclude from QA
4,Mature Themes and Adult Content,Mature Themes and Adult Content,Mature Themes and Adult Content,Mature Themes and Adult Content,Mature Themes and Adult Content,Mature Themes and Adult Content,"louvecchia, creamyjuicy, ahloubaby, emilycocea, mym_antille_guyane, paolalocatelli, 310lexa, 972_nude, melaniemartialdc, anaisbragadias",10,exclude from QA


## Create new df_clean 

- Add list of subreddits to target-CLEAN, b/c we'll need it for rating final
- Add new columns & update order


### Copy baseline cols for clean

In [ ]:
col_model_sort_order = 'model_sort_order'

# create new col as int for label so we can add a color scale when doing QA
col_new_cluster_val_int = 'cluster_label_int'
df_labels_target_dynamic_raw[col_new_cluster_val_int] = (
    df_labels_target_dynamic_raw[col_new_cluster_val]
    .str[-4:].astype(int)
)

# create new column for reddit link, makes QA easier
col_link_to_sub = 'link_to_sub'
link_prefix_ = 'www.reddit.com/r/'
df_labels_target_dynamic_raw[col_link_to_sub] = link_prefix_ + df_labels_target_dynamic_raw['subreddit_name']

# create lookup for rating col, might need to refresh it later in google sheets, though
col_rated_e = 'rated E'
# need to make it boolean for checkbox thing to work
df_labels_target_dynamic_raw[col_rated_e] = (
    df_labels_target_dynamic_raw['rating_short'] == 'E'
)

# Set expected col order
l_cols_clean_final_for_qa = [
    'subreddit_id',
    'subreddit_name',
    col_new_cluster_val_int,
    col_new_cluster_topic_mix,
    
    # insert inputs for QA cols
    'not country relevant',
    col_rated_e,
    'relevant to cluster/ other subreddits in cluster',
    'safe to show in relation to cluster',
    col_link_to_sub,

    # cols for notes
    'country relevance notes',
    'rating or cluster notes',
    #  # 'cluster relevance notes',

    col_subs_in_cluster_count,
    col_list_cluster_names,

    'allow_discovery',
    'users_l7',
    'total_users_in_subreddit_l28',
    'rating_name',

    'over_18',
    'rating_short',
    'primary_topic',
    col_subreddit_topic_mix,
    # why did a sub get marked as geo or culturally relevant?
    #  can use them to sort
    'geo_relevance_default',
    'relevance_percent_by_subreddit',
    'relevance_percent_by_country_standardized',
    'b_users_percent_by_subreddit',
    'e_users_percent_by_country_standardized',
    'd_users_percent_by_country_rank',
    col_model_sort_order,

    'posts_for_modeling_count',
    'c_users_percent_by_country',
    'users_in_subreddit_from_country_l28',
    'total_users_in_country_l28',
    
    col_new_cluster_val,
    col_new_cluster_name,
    col_new_cluster_prim_topic,
]

# copy existing columns from raw + 
l_cols_clean_existing = [c for c in l_cols_clean_final_for_qa if c in df_labels_target_dynamic_raw.columns]
l_cols_clean_new = [c for c in l_cols_clean_final_for_qa if c not in df_labels_target_dynamic_raw.columns]

# reorder cols in raw DF so it's easier to switch between the two tabs
df_labels_target_dynamic_raw = df_labels_target_dynamic_raw[
    reorder_array(l_cols_clean_existing, df_labels_target_dynamic_raw.columns)
]

# only copy subs that aren't in excluded clusters!!
df_labels_target_dynamic_clean = (
    df_labels_target_dynamic_raw
    [df_labels_target_dynamic_raw[col_exclude_from_qa] != val_exclude_from_qa_]
    [l_cols_clean_existing]
    .copy()
)
# Add new columns and initialize them with empty strings
df_labels_target_dynamic_clean[l_cols_clean_new] = ''

# check num of orphans for clean
display(style_df_numeric(
    get_dynamic_cluster_summary(
        df_labels_target_dynamic_clean,
        return_dict=False,
    ),
    rename_cols_for_display=True,
).set_caption(f"Summary for df-CLEAN"))

# re-order & rename the columns so their easier to see in google sheets
# Sorty by cluster label b/c sometimes a sub won't be clustered dynamically next to closest neighbors!
df_labels_target_dynamic_clean = (
    df_labels_target_dynamic_clean[l_cols_clean_final_for_qa]
    .sort_values(by=[col_new_cluster_val, col_model_sort_order], ascending=True)
    # do final renaming only when saving, otherwise, it's a pain to adjust or look things up?
    .rename(columns={c: c.replace('_', ' ') for c in l_cols_clean_final_for_qa[5:]})
)

print(f"{len(l_cols_clean_final_for_qa)} <- expected final col count")
print(f"{df_labels_target_dynamic_raw.shape} <- df raw shape")
print(f"{df_labels_target_dynamic_clean.shape} <- df clean shape")

,cluster count,num orphan subreddits,num subreddits per cluster mean,num subreddits per cluster median,num clusters with mature primary topic,cluster ids with orphans
0,40,0,8.25,8.00,0,


35 <- expected final col count
(378, 131) <- df raw shape
(330, 35) <- df clean shape


In [ ]:
df_labels_target_dynamic_clean.iloc[:5, :24]

,subreddit_id,subreddit_name,cluster_label_int,cluster_topic_mix,not country relevant,rated E,relevant to cluster/ other subreddits in cluster,safe to show in relation to cluster,link to sub,country relevance notes,rating or cluster notes,subs in cluster count,list cluster subreddit names,allow discovery,users l7,total users in subreddit l28,rating name,over 18,rating short,primary topic,subreddit full topic mix,geo relevance default,relevance percent by subreddit,relevance percent by country standardized
73,t5_2v7bv7,calyxos,6,Technology,,True,,,www.reddit.com/r/calyxos,,,3,"calyxos, pedale, thegreatgasly",None,6388,25342,Everyone,None,E,Technology,Technology,False,False,True
74,t5_2z6r0,pedale,6,Technology,,True,,,www.reddit.com/r/pedale,,,3,"calyxos, pedale, thegreatgasly",None,2752,9195,Everyone,None,E,None,Cars and Motor Vehicles | Hobbies,True,True,True
75,t5_4as8z,thegreatgasly,6,Technology,,True,,,www.reddit.com/r/thegreatgasly,,,3,"calyxos, pedale, thegreatgasly",t,550,1303,Everyone,f,E,Celebrity,Cars and Motor Vehicles | Sports,False,False,True
48,t5_fucxb,traefik,25,Programming,,True,,,www.reddit.com/r/traefik,,,8,"traefik, ansible, symfony, airtable, powershell, ada, numworks, programmation",t,5695,19298,Everyone,f,E,Technology,Programming | Technology,False,False,True
49,t5_2x7n9,ansible,25,Programming,,True,,,www.reddit.com/r/ansible,,,8,"traefik, ansible, symfony, airtable, powershell, ada, numworks, programmation",None,25169,85055,Everyone,None,E,Technology,Programming | Technology,False,False,True


## Create target to target list for FPR example

In [ ]:
%%time

df_target_to_target_list = convert_distance_or_ab_to_list_for_fpr(
    df_labels_target_dynamic_clean.rename(columns={c: c.replace(' ', '_') for c in df_labels_target_dynamic_clean.columns}),
    convert_to_ab=True,
    col_counterpart_count='counterpart_count',
    col_list_cluster_names='list_cluster_subreddit_names',
    col_list_cluster_ids='list_cluster_subreddit_ids',
    l_cols_for_seeds=None,
    l_cols_for_clusters=None,
    col_new_cluster_val=col_new_cluster_val,
    col_new_cluster_name=col_new_cluster_name,
    col_new_cluster_prim_topic=col_new_cluster_prim_topic,
    verbose=False,
)
df_target_to_target_list.shape

  (2634, 9) <- df_ab.shape after removing matches to self
  (330, 7) <- df_a_to_b.shape
CPU times: user 37 ms, sys: 0 ns, total: 37 ms
Wall time: 37 ms


In [ ]:
df_target_to_target_list.iloc[:5, :11]

,subreddit_name_seed,subreddit_id_seed,cluster_label,cluster_label_k,counterpart_count,list_cluster_subreddit_names,list_cluster_subreddit_ids
0,traefik,t5_fucxb,0006-0008-0013-0016-0016-0020-0021-0025,k_0118_label,7,"ansible, symfony, airtable, powershell, ada, numworks, programmation","t5_2x7n9, t5_2r358, t5_36dkt, t5_2qo1o, t5_2qi8f, t5_3fwdt, t5_2t5jj"
1,ansible,t5_2x7n9,0006-0008-0013-0016-0016-0020-0021-0025,k_0118_label,7,"traefik, symfony, airtable, powershell, ada, numworks, programmation","t5_fucxb, t5_2r358, t5_36dkt, t5_2qo1o, t5_2qi8f, t5_3fwdt, t5_2t5jj"
2,symfony,t5_2r358,0006-0008-0013-0016-0016-0020-0021-0025,k_0118_label,7,"traefik, ansible, airtable, powershell, ada, numworks, programmation","t5_fucxb, t5_2x7n9, t5_36dkt, t5_2qo1o, t5_2qi8f, t5_3fwdt, t5_2t5jj"
3,airtable,t5_36dkt,0006-0008-0013-0016-0016-0020-0021-0025,k_0118_label,7,"traefik, ansible, symfony, powershell, ada, numworks, programmation","t5_fucxb, t5_2x7n9, t5_2r358, t5_2qo1o, t5_2qi8f, t5_3fwdt, t5_2t5jj"
4,powershell,t5_2qo1o,0006-0008-0013-0016-0016-0020-0021-0025,k_0118_label,7,"traefik, ansible, symfony, airtable, ada, numworks, programmation","t5_fucxb, t5_2x7n9, t5_2r358, t5_36dkt, t5_2qi8f, t5_3fwdt, t5_2t5jj"


# Export data

Note that we have to use `fillna('')`

Otherwise, we'll get errors because the gspread library doesn't know how to handle `pd.NaN` or `np.Nan` (nulls).

## Define variables to create/access google sheet doc & worksheets

In [ ]:
gspread.__version__

'4.0.1'

In [ ]:
# # %%time
os.environ['GOOGLE_CLOUD_PROJECT'] = 'data-prod-165221'
country_name_sheet_ = 'France'
target_abbrev_ = 'fr'
GSHEET_KEY = '1RicLwjnVxArCiPqoVUPEIbFnruL5m7VBL89ZXBCix7c'
GSHEET_NAME = f'i18n {country_name_sheet_} subreddits and clusters - model v0.4.1'


d_wsh_names = {
    'qa_ready': {
        'name': 'subs_need_to_be_rated',
    },
    'sub_raw': {
        'name': 'raw_data_per_subreddit',
    },
    'clusters_t2t_list_raw': {
        'name': f'raw_clusters_list_{target_abbrev_}_{target_abbrev_}',
    },
    'clusters_t2t_fpr_raw': {
        'name': f'raw_clusters_fpr_{target_abbrev_}_{target_abbrev_}',
    }
}

if GSHEET_KEY is not None:
    sh = gc.open_by_key(GSHEET_KEY)
    print(f"Opening google worksheet: {GSHEET_NAME} ...")
else:
    print(f"** Creating google worksheet: {GSHEET_NAME} ...")
    sh = gc.create(GSHEET_NAME)

# create worksheets:
for _, d_ in d_wsh_names.items():
    sh_name = d_['name']
    try:
        d_['worksheet'] = sh.worksheet(sh_name)
        print(f"Opening tab/sheet: {sh_name} ...")
    except Exception as e:
        print(f"** Creating tab/sheet: {sh_name} ...")
        d_['worksheet'] = sh.add_worksheet(sh_name, rows=5, cols=5)


Opening google worksheet: i18n France subreddits and clusters - model v0.4.1 ...
Opening tab/sheet: subs_need_to_be_rated ...
Opening tab/sheet: raw_data_per_subreddit ...
Opening tab/sheet: raw_clusters_list_fr_fr ...
Opening tab/sheet: raw_clusters_fpr_fr_fr ...


## Save: Clean sheet to rate

In [ ]:
d_wsh_names.keys()

dict_keys(['qa_ready', 'sub_raw', 'clusters_t2t_list_raw', 'clusters_t2t_fpr_raw'])

In [ ]:
%%time
(
    d_wsh_names['qa_ready']['worksheet']
    .update([df_labels_target_dynamic_clean.columns.values.tolist()] + 
             df_labels_target_dynamic_clean.fillna('').values.tolist())
)

CPU times: user 11.8 ms, sys: 2 ms, total: 13.8 ms
Wall time: 362 ms


{'spreadsheetId': '1RicLwjnVxArCiPqoVUPEIbFnruL5m7VBL89ZXBCix7c',
 'updatedCells': 11585,
 'updatedColumns': 35,
 'updatedRange': 'subs_need_to_be_rated!A1:AI331',
 'updatedRows': 331}

## Save: df cluster for humans

In [ ]:
%%time

(
    d_wsh_names['clusters_t2t_list_raw']['worksheet']
    .update(
        [df_cluster_for_humans.columns.values.tolist()] + 
        df_cluster_for_humans.fillna('').values.tolist()
    )
)

CPU times: user 6.63 ms, sys: 43 µs, total: 6.68 ms
Wall time: 237 ms


{'spreadsheetId': '1RicLwjnVxArCiPqoVUPEIbFnruL5m7VBL89ZXBCix7c',
 'updatedCells': 343,
 'updatedColumns': 7,
 'updatedRange': 'raw_clusters_list_fr_fr!A1:G49',
 'updatedRows': 49}

## Save: target raw dynamic


In [ ]:
l_cols_to_drop = (
    ['table_creation_date'] +
    [c for c in df_labels_target_dynamic_raw.columns if c.endswith('_nested')]
)
print(len(l_cols_to_drop))
# df_labels_target_dynamic_raw.columns.to_list()

45


In [ ]:
%%time
(
    d_wsh_names['sub_raw']['worksheet']
    .update([df_labels_target_dynamic_raw.drop(l_cols_to_drop, axis=1).columns.values.tolist()] + 
             df_labels_target_dynamic_raw.drop(l_cols_to_drop, axis=1).fillna('').values.tolist())
)

CPU times: user 31.5 ms, sys: 148 µs, total: 31.6 ms
Wall time: 714 ms


{'spreadsheetId': '1RicLwjnVxArCiPqoVUPEIbFnruL5m7VBL89ZXBCix7c',
 'updatedCells': 32594,
 'updatedColumns': 86,
 'updatedRange': 'raw_data_per_subreddit!A1:CH379',
 'updatedRows': 379}

## Save: FPR target-2-target as list

Even though data isn't fully ready, want to have the output ready to make sure it's in the right format that we need.

In [ ]:
df_target_to_target_list.iloc[:5, :5]

,subreddit_name_seed,subreddit_id_seed,cluster_label,cluster_label_k,counterpart_count
0,traefik,t5_fucxb,0006-0008-0013-0016-0016-0020-0021-0025,k_0118_label,7
1,ansible,t5_2x7n9,0006-0008-0013-0016-0016-0020-0021-0025,k_0118_label,7
2,symfony,t5_2r358,0006-0008-0013-0016-0016-0020-0021-0025,k_0118_label,7
3,airtable,t5_36dkt,0006-0008-0013-0016-0016-0020-0021-0025,k_0118_label,7
4,powershell,t5_2qo1o,0006-0008-0013-0016-0016-0020-0021-0025,k_0118_label,7


In [ ]:
%%time

(
    d_wsh_names['clusters_t2t_fpr_raw']['worksheet']
    .update(
        [df_target_to_target_list.columns.values.tolist()] + 
        df_target_to_target_list.fillna('').values.tolist()
    )
)

CPU times: user 9.43 ms, sys: 11 µs, total: 9.44 ms
Wall time: 267 ms


{'spreadsheetId': '1RicLwjnVxArCiPqoVUPEIbFnruL5m7VBL89ZXBCix7c',
 'updatedCells': 2317,
 'updatedColumns': 7,
 'updatedRange': 'raw_clusters_fpr_fr_fr!A1:G331',
 'updatedRows': 331}

## We can read the data back to confirm it's as expected

In [ ]:
# Here's how to get the records as a dataframe
pd.DataFrame(
    d_wsh_names['sub_raw']['worksheet'].get_all_records()
).iloc[:5, :15]

,subreddit_id,subreddit_name,cluster_label_int,cluster_topic_mix,rated E,link_to_sub,subs_in_cluster_count,list_cluster_subreddit_names,allow_discovery,users_l7,total_users_in_subreddit_l28,rating_name,over_18,rating_short,primary_topic
0,t5_3wcby1,influenceusestvr,1,Mature Themes and Adult Content,FALSE,www.reddit.com/r/influenceusestvr,3,"influenceusestvr, alyssamilano, edithbhr",,16151,48267,,f,,Mature Themes and Adult Content
1,t5_2oq21o,louvecchia,3,Mature Themes and Adult Content,FALSE,www.reddit.com/r/louvecchia,10,"louvecchia, creamyjuicy, ahloubaby, emilycocea, mym_antille_guyane, paolalocatelli, 310lexa, 972_nude, melaniemartialdc, anaisbragadias",,222,848,,f,,
2,t5_4148ft,creamyjuicy,3,Mature Themes and Adult Content,FALSE,www.reddit.com/r/creamyjuicy,10,"louvecchia, creamyjuicy, ahloubaby, emilycocea, mym_antille_guyane, paolalocatelli, 310lexa, 972_nude, melaniemartialdc, anaisbragadias",,5088,14148,,f,,Mature Themes and Adult Content
3,t5_5blr1t,ahloubaby,3,Mature Themes and Adult Content,FALSE,www.reddit.com/r/ahloubaby,10,"louvecchia, creamyjuicy, ahloubaby, emilycocea, mym_antille_guyane, paolalocatelli, 310lexa, 972_nude, melaniemartialdc, anaisbragadias",,454,1176,,f,,
4,t5_4fknoz,emilycocea,3,Mature Themes and Adult Content,TRUE,www.reddit.com/r/emilycocea,10,"louvecchia, creamyjuicy, ahloubaby, emilycocea, mym_antille_guyane, paolalocatelli, 310lexa, 972_nude, melaniemartialdc, anaisbragadias",,2737,7755,Everyone,f,E,Celebrity


# Appendix


## Additional checks on cluster depth

In [ ]:
print(df_labels_target_dynamic_raw['cluster_label'].nunique())
display(
    value_counts_and_pcts(
        df_labels_target_dynamic_raw,
        ['cluster_label'],
        top_n=10,
    )
)
value_counts_and_pcts(
    df_labels_target_dynamic_raw,
    ['cluster_label'],
    top_n=None,
    return_df=True
)['count'].describe()

53


,count,percent,cumulative_percent
cluster_label,,,
0006-0008-0013-0017-0017-0021-0022-0026,17,4.5%,4.5%
0011-0018-0033-0048-0051-0064-0068-0096,13,3.4%,7.9%
0007-0011-0019-0026-0026-0035-0036-0046-0132-0265-0393-0443-0642-0756-0910-0968-1171-1301-1401-1528-1595-1620,12,3.2%,11.1%
0001-0001-0002-0002-0002-0003-0003-0003,10,2.6%,13.8%
0010-0017-0030-0040-0043-0054-0057,10,2.6%,16.4%
0011-0018-0032-0043-0046-0059-0063,9,2.4%,18.8%
0001-0001-0003-0003-0003-0004-0005-0005,9,2.4%,21.2%
0010-0017-0030-0040-0043-0054-0057-0081-0226-0449-0648-0725-1051-1236-1484-1580-1888-2096-2264-2456-2562-2597,9,2.4%,23.5%
0010-0017-0031-0041-0044-0055-0058-0083,9,2.4%,25.9%


count    53.000000
mean      7.132075
std       2.696420
min       1.000000
25%       6.000000
50%       7.000000
75%       8.000000
max      17.000000
Name: count, dtype: float64

### How deep are the clusters?

Looks like some peaks around 100, 300, 1k, and 4k clusters.

In [ ]:
print(df_labels_target_dynamic_raw[col_new_cluster_name].nunique())
value_counts_and_pcts(
    df_labels_target_dynamic_raw,
    [col_new_cluster_name],
    top_n=None,
    sort_index=True,
)

15


,count,percent,cumulative_percent
cluster_label_k,,,
k_0013_label,47,12.4%,12.4%
k_0023_label,14,3.7%,16.1%
k_0041_label,22,5.8%,22.0%
k_0059_label,7,1.9%,23.8%
k_0063_label,6,1.6%,25.4%
k_0085_label,39,10.3%,35.7%
k_0118_label,112,29.6%,65.3%
k_0320_label,47,12.4%,77.8%
k_0657_label,14,3.7%,81.5%


In [ ]:
# style_df_numeric(
#     df_labels_target.tail(10),
#     # rename_cols_for_display=True,
#     l_bar_simple=[c for c in df_labels_target.columns if '_label' in c]
# )

# Filter out subs [extra]

UPDATE: For now let's include all the subreddits for QA because this list could potentially help us rate/flag subreddits that aren't rated or mis-rated and have a lot of traffic.

Now that we have even more clusters (over 3,000), it's harder to figure out where to set the threshold for clusters to exclude.

--- 

The main use case for now are SFW subs, we could save some QA time by excluding these subs:
- Exclude NSFW clusters
- Exclude place subs


~We'll use the cluster labels to discard subreddits because~
- many of the DE subreddits don't have a `primary_topic`
- if the majority of subs for a subreddits are NSFW, then we wouldn't want to recommend those anyway

In [ ]:
# # we can see that the subreddit count changes as we go 
# #  from shallow to deeper cluster counts
# value_counts_and_pcts(
#     df_labels_target['k_0118_majority_primary_topic'],
#     top_n=15,
#     reset_index=True,
#     add_col_prefix=False,
#     count_type='subreddits',
#     return_df=False,
# )

In [ ]:
# value_counts_and_pcts(
#     df_labels_target['k_3145_majority_primary_topic'],
#     top_n=15,
#     reset_index=True,
#     add_col_prefix=False,
#     count_type='subreddits',
#     return_df=False,
# )

In [ ]:
# value_counts_and_pcts(
#     df_labels_target['k_3927_majority_primary_topic'],
#     top_n=15,
#     reset_index=True,
#     add_col_prefix=False,
#     count_type='subreddits',
#     return_df=False,
# )

In [ ]:
# # And the count is slightly different from the known primary topics
# #  We still have a large number of subs w/o a primary topic
# value_counts_and_pcts(
#     df_labels_target['primary_topic'],
#     count_type='subreddits',
#     reset_index=True,
#     add_col_prefix=False,
# )

In [ ]:
print(f"{df_labels_target.shape} <- Shape before filtering")

l_manual_subs_to_remove = [
    'sexmeets1', 'fuck',
]
col_cluster_filter = 'k_3145_majority_primary_topic'
df_labels_target_clean = (
    df_labels_target[df_labels_target[col_cluster_filter] != 'Mature Themes and Adult Content']
)
print(f"{df_labels_target_clean.shape} <- Shape after dropping NSFW clusters")

l_sensitive_topics = [
    'Military', 'Gender', 'Addiction Support',
    'Medical and Mental Health', 'Sexual Orientation',
    'Culture, Race, and Ethnicity',
]
df_labels_target_clean = (
    df_labels_target_clean[
        ~df_labels_target_clean[col_cluster_filter].isin(l_sensitive_topics)
    ]
)
print(f"{df_labels_target_clean.shape} <- Shape after dropping Sensitive clusters")

df_labels_target_clean = (
    df_labels_target_clean[
        ~df_labels_target_clean['primary_topic'].isin(l_sensitive_topics)
    ]
)
print(f"{df_labels_target_clean.shape} <- Shape after dropping SENSITIVE subreddits")


df_labels_target_clean = (
    df_labels_target_clean[
        ~df_labels_target_clean['subreddit_name'].isin(l_manual_subs_to_remove)
    ]
)
print(f"{df_labels_target_clean.shape} <- Shape after dropping Manual list of subreddits")

print(f"  ** TODO: instead of excluding place subs, add logic to map hierarchy **")
# df_labels_target_clean = (
#     df_labels_target_clean[df_labels_target_clean['primary_topic'] != 'Place']
# )
# print(f"{df_labels_target_clean.shape} <- Shape after dropping Place subreddits")

(378, 74) <- Shape before filtering
(335, 74) <- Shape after dropping NSFW clusters
(327, 74) <- Shape after dropping Sensitive clusters
(326, 74) <- Shape after dropping SENSITIVE subreddits
(326, 74) <- Shape after dropping Manual list of subreddits
  ** TODO: instead of excluding place subs, add logic to map hierarchy **
